In [17]:
# !pip3 install git+https://github.com/rongardF/tvdatafeed.git@e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
from tvDatafeed import TvDatafeed, Interval

In [18]:
from helpers_db import get_engine, run_sql
engine = get_engine()

In [19]:
from helpers_queries import queries
assets = None
with engine.begin() as conn:
  assets = run_sql(conn, queries['list_assets_today']).mappings().all()
assets

[{'symbol': 'AAOI', 'exchange': 'NASDAQ', 'category': 'stock', 'bars_d': 1, 'bars_4h': 8034},
 {'symbol': 'AAPL', 'exchange': 'NASDAQ', 'category': 'stock', 'bars_d': 1, 'bars_4h': 4926},
 {'symbol': 'ALB', 'exchange': 'NYSE', 'category': 'stock', 'bars_d': 1, 'bars_4h': 8832},
 {'symbol': 'BABA', 'exchange': 'NYSE', 'category': 'stock', 'bars_d': 1, 'bars_4h': 3828},
 {'symbol': 'CI', 'exchange': 'NYSE', 'category': 'stock', 'bars_d': 1, 'bars_4h': 5136},
 {'symbol': 'CLS', 'exchange': 'NYSE', 'category': 'stock', 'bars_d': 1, 'bars_4h': 8916},
 {'symbol': 'CRDF', 'exchange': 'NASDAQ', 'category': 'stock', 'bars_d': 1, 'bars_4h': 1650},
 {'symbol': 'CSCO', 'exchange': 'NASDAQ', 'category': 'stock', 'bars_d': 1, 'bars_4h': 8874},
 {'symbol': 'DBK', 'exchange': 'XETR', 'category': 'stock', 'bars_d': 1, 'bars_4h': 4716},
 {'symbol': 'ESEA', 'exchange': 'NASDAQ', 'category': 'stock', 'bars_d': 1, 'bars_4h': 5556},
 {'symbol': 'GCT', 'exchange': 'NASDAQ', 'category': 'stock', 'bars_d': 1, 

In [20]:
from helpers_db import postgres_upsert

tv = TvDatafeed()

def fetch_ticks(symbol, exchange, interval=Interval.in_daily, bars=100, extended_session=True):
  df = tv.get_hist(
      symbol=symbol,
      exchange=exchange,
      interval=interval,
      n_bars=bars,
      extended_session=extended_session,
  )
  return df

def fix_symbol(eachange_symbol):
  return eachange_symbol.split(':')[-1] if ':' in eachange_symbol else eachange_symbol

def normalize_dataframe(df):
  df.reset_index(inplace=True)
  df['symbol'] = df['symbol'].apply(fix_symbol)
  if 'datetime' in list(df.columns.values):
    df['dt'] = df['datetime']
    df.drop(columns=['datetime'], inplace=True)
  return df

hourly  = {'interval': Interval.in_4_hour, 'bars': {'default': 10, 'query': False, 'field': 'bars_4h'}, 'table': 'ticks_4h', 'extended_session': True}
daily   = {'interval': Interval.in_daily, 'bars': {'default': 10, 'query': False, 'field': 'bars_d'}, 'table': 'ticks_1d', 'extended_session': True}
weekly  = {'interval': Interval.in_weekly, 'bars': {'default': 10, 'query': False, 'field': 'bars_w'}, 'table': 'ticks_1w', 'extended_session': True}
period  = daily

print(f"Assets count: {len(assets)}")
print(f"Running table: {period['table']} {period['interval']} bars_default: {period['bars']['default']}")
for asset in assets:
  symbol      = asset['symbol']
  exchange    = asset['exchange']
  bars_count  = asset[period['bars']['field']] if period['bars']['query'] else period['bars']['default'] 
  try:
    
    bars = bars_count if period['bars']['query'] else period['bars']['default']
    print(f"Processing {exchange}:{symbol}::{bars_count}")
    df = fetch_ticks(symbol=symbol, exchange=exchange, bars=bars, interval=period['interval'], extended_session=period['extended_session'])
    df = normalize_dataframe(df)
    df.to_sql(period['table'], engine, if_exists='append', index=False, method=postgres_upsert)
    
  except Exception as e:
    print(f"Exception {exchange}:{symbol}")
    print(str(e))
  # break


Assets count: 31
Running table: ticks_1d Interval.in_daily bars_default: 10
Processing NASDAQ:AAOI::10
Processing NASDAQ:AAPL::10
Processing NYSE:ALB::10
Processing NYSE:BABA::10
Processing NYSE:CI::10
Processing NYSE:CLS::10
Processing NASDAQ:CRDF::10
Processing NASDAQ:CSCO::10
Processing XETR:DBK::10
Processing NASDAQ:ESEA::10
Processing NASDAQ:GCT::10
Processing NASDAQ:GOOG::10
Processing NASDAQ:INMD::10
Processing NYSE:JPM::10
Processing NYSE:KO::10
Processing EURONEXT:LQQ::10
Processing NASDAQ:META::10
Processing NASDAQ:MSFT::10
Processing NASDAQ:NKTX::10
Processing NASDAQ:NVDA::10
Processing NASDAQ:ON::10
Processing NYSE:PAYC::10
Processing NASDAQ:PYPL::10
Processing NASDAQ:SBUX::10
Processing NASDAQ:SEDG::10
Processing NYSE:T::10
Processing NASDAQ:WDAY::10
Processing NYSE:HEI::10
Processing NASDAQ:SURG::10
Processing NASDAQ:MSTR::10
Processing NASDAQ:AMZN::10
